In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

evalTime = 25.

In [39]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst

    
def get_last_line_as_numpy_array(file_path, dtype = float, verbose = False):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        
        try:
            last_line = lines[-1].strip()
        except:
            print('timeLine',file_path, len(lines))
            raise Exception

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)
        if verbose:
            print(file_path, len(array_from_last_line), end=", ")

        return array_from_last_line

def getCylIdx(path2file):
    fileName = "rhizoSegsId.txt"
    fullpath = path2file+fileName
    rhizoSegsId = get_last_line_as_numpy_array(fullpath, dtype = int, 
                                               verbose = False)
    print('rhizoSegsId',max(rhizoSegsId))
    return rhizoSegsId
    
def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names

# cylinder max konz (per cell)
def getData_(cid,gId,path2file, verbose = False):
    if verbose :
        print('getData_',cid,gId,path2file, volIdx,coordId)
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return cVol
    elif cid <= nKonz:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName,
                                                 dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return cVol
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = Q1/cVol #mol/cm3 or cm3/cm3
        return konz
    elif cid == coordId:
        fileName = "cyl_val/Cyl_coord_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName,
                                          dtype = float)

        return Q1
    raise Exception

nKonz = 2
volIdx=3
lenIdx=4
stIdx=5
orgIdId=6
relLenId=7
coordId = 8

nToGet = coordId+1#elements + 3d coordinates of y , node+vol+len+st 

numPset = 99
exceptPset = []
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    
    for pSet in allPSets:
        print(pSet,end =", ")
        
        path2file = 'newMucil/none_1476_'+str(pSet)+'_10to11_20mn_0s_10/'

        rr = getCylIdx(path2file)

        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, 
                                        gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits

        Ginits = np.array([getData_( coordId, gId,path2file
                                   ) for gId in rr])
        GiniAll[coordId][pSet] = Ginits


        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float, 
                                            verbose = False)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float, 
                                           verbose = False)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]

        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int, 
                                                    verbose = False)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int, 
                                                    verbose = False)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int,
                                                  verbose = False)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int, 
                                               verbose = False)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", 
                                               dtype =int,
                                               verbose = False)
        id_roots = id_orgs[np.where(ot_orgs==2)]

        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])

        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))

        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])


        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens


    return GiniAll #cid pSet rr
    

In [40]:

GiniAll = getData()

0, rhizoSegsId 227
( 196 ), 1, rhizoSegsId 227
( 196 ), 2, rhizoSegsId 227
( 196 ), 3, rhizoSegsId 227
( 196 ), 4, rhizoSegsId 229
( 198 ), 5, rhizoSegsId 227
( 196 ), 6, rhizoSegsId 227
( 196 ), 7, rhizoSegsId 229
( 198 ), 8, rhizoSegsId 227
( 196 ), 9, rhizoSegsId 227
( 196 ), 10, rhizoSegsId 227
( 196 ), 11, rhizoSegsId 227
( 196 ), 12, rhizoSegsId 227
( 196 ), 13, rhizoSegsId 227
( 196 ), 14, rhizoSegsId 227
( 196 ), 15, rhizoSegsId 228
( 197 ), 16, rhizoSegsId 227
( 196 ), 17, rhizoSegsId 227
( 196 ), 18, rhizoSegsId 227
( 196 ), 19, rhizoSegsId 228
( 197 ), 20, rhizoSegsId 227
( 196 ), 21, rhizoSegsId 227
( 196 ), 22, rhizoSegsId 227
( 196 ), 23, rhizoSegsId 227
( 196 ), 24, rhizoSegsId 228
( 197 ), 25, rhizoSegsId 227
( 196 ), 26, rhizoSegsId 227
( 196 ), 27, rhizoSegsId 227
( 196 ), 28, rhizoSegsId 227
( 196 ), 29, rhizoSegsId 227
( 196 ), 30, rhizoSegsId 227
( 196 ), 31, rhizoSegsId 227
( 196 ), 32, rhizoSegsId 227
( 196 ), 33, rhizoSegsId 227
( 196 ), 34, rhizoSegsId 227
( 19

In [41]:
GiniAll[coordId][0][0]

array([0.05499005, 0.06596616, 0.07913313, 0.09492826, 0.11387611,
       0.136606  , 0.16387281, 0.19658214, 0.2358203 ])

In [42]:
df = pd.DataFrame(GiniAll)
df.to_pickle('./pset99_allcell.pkl')

In [20]:
# get param set
outputvalsname =  ['wat','cs','vol','lenSeg','st','orgId','relLen','relcoord']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname# + ['scenario']
df = pd.DataFrame(columns=column_names)

for pset in range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_df = pd.DataFrame()

    for jj, outVn in enumerate(outputvalsname):
        rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))

    rows_df['pSet'] = pset

    df = pd.concat([df, rows_df], ignore_index=True)



ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [21]:
GiniAll[jj][pset]

array([[0.00073402, 0.00105629, 0.00152004, ..., 0.00651858, 0.00938053,
        0.013499  ],
       [0.00011927, 0.00017239, 0.00024916, ..., 0.00108735, 0.00157159,
        0.00227149],
       [0.00014264, 0.00021852, 0.00033476, ..., 0.00184375, 0.00282453,
        0.00432704],
       ...,
       [0.00030158, 0.00063353, 0.00133088, ..., 0.02591889, 0.05444852,
        0.11438157],
       [0.00015471, 0.00024237, 0.0003797 , ..., 0.00228717, 0.00358313,
        0.00561342],
       [0.00015471, 0.00024237, 0.0003797 , ..., 0.00228717, 0.00358313,
        0.00561342]])

In [30]:
df.to_pickle('./4paramTemp.pkl')

In [17]:
nToGet = coordId